Assignment: In the following code already discussed in the practical session, you have to achieve atleast 80% accuracy. You are allowed to make use of any classifier available in the sklearn library or you can apply ANN too.
Note: Features should not be changed.(They should be LBP only)

**Let's discuss the code**

Following code discusses binary classification problem.

It classifies the image into two classes Airplane or Bird.

The dataset used for training and testing contains 500 images of each class.

Samples are taken from CIFAR-10 dataset.

*******************************************************************************************************************************

**Import the necessary packages**

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from skimage.color import rgb2gray
import math

Using TensorFlow backend.


**Following function reads images from the folder, finds vertical and horizontal edges using prewitt operator on each image, and edges are taken as features.**

In [2]:
def im2lbp(im):
    # there is no histogram computation
    im = np.array(im)
    m=im.shape[0]
    n=im.shape[1]
   
    t = np.zeros(8,dtype=np.uint8)
    Lbpdfeat = []
   
    for x in range (1,m-1):
         
        for y in range (1,n-1):
            #i=0
            t[0]=(im[x][y+1]>im[x][y])
            t[1]=(im[x-1][y+1]>im[x][y])
            t[2]=(im[x-1][y]>im[x][y])
            t[3]=(im[x-1][y-1]>im[x][y])
            t[4]=(im[x][y-1]>im[x][y])
            t[5]=(im[x+1][y-1]>im[x][y])
            t[6]=(im[x+1][y]>im[x][y])
            t[7]=(im[x+1][y+1]>im[x][y])
   
            d=0;
            for z in range (0,8):
               if(t[z]==1):
                d=d+math.pow(2,z)
            Lbpdfeat.append(d)
        #print(Lbpdfeat)
    return Lbpdfeat

In [3]:
def flatten_images(folder):
    features=[] # list contatining features of all images rowwise 
    for filename in os.listdir(folder):
        img=plt.imread(folder+filename)
        img=rgb2gray(img)
        lbpfeat=im2lbp(img)
        hist=np.histogram(lbpfeat,256)  # 8 bit features are extracted because of 8 neighbours 
        features.append(hist[0])
    return(features)

In [4]:
airplane_data=flatten_images('data_500\\airplane\\') # calling the function above to compute airplane features
airplane_data=np.array(airplane_data) # converting list into numpy array
airplane_data.shape  # print shape of array
# In the output output : rows count shows the number of images and columns count is the number of features

(500, 256)

In [5]:
# same as above but for bird data
bird_data=flatten_images('data_500\\bird\\')
bird_data=np.array(bird_data)
bird_data.shape

(500, 256)

**Create labels for the airplane and bird feature vectors** 

In [6]:
airplane_labels=np.ones((len(airplane_data),1)) # np.ones creates an array of ones (where 1 is the label) with only one column and number of rows same as airplane features
bird_labels=np.zeros((len(bird_data),1))# np.zeros creates an array of ones (where 1 is the label) with only one column and number of rows same as airplane features

**Create feature vector and corresponding label vector containing both the classes airplane and bird**

In [7]:
labels=np.concatenate((airplane_labels,bird_labels),axis=0)
features=np.concatenate((airplane_data,bird_data),axis=0)
np.unique(features)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 149, 150, 151, 152, 153, 156, 158, 159, 160,
       161, 162, 163, 165, 166, 167, 168, 170, 174, 175, 178, 180, 182,
       183, 185, 188, 191, 193, 194, 196, 199, 201, 203, 204, 20

**Normalize the data**

In [41]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(features)
normalized_features=scaler.transform(features)
np.unique(normalized_features)

array([-2.54065563, -2.37246155, -2.20426748, ..., 14.74577722,
       16.08152308, 16.70611683])

In [42]:
from imblearn.over_sampling import SMOTE, ADASYN
X_resampled, y_resampled = ADASYN().fit_resample(features, labels)

In [43]:
from sklearn.model_selection import train_test_split
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(features,labels,test_size=0.4)

In [44]:
# from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(max_depth=500, random_state=100)

# Train the model using the training sets
model.fit(X_train,y_train)
# Test the model using the training sets
y_pred = model.predict(X_test)

C:\Users\Anaya\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [45]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


Accuracy: 0.71


**ANN classifier**

In [63]:
from keras.models import Sequential          #  Sequential class is created and model layers are created and added to it
from keras.layers import Activation, Dense,Dropout   # 
from keras import optimizers
from keras import applications
from keras.models import Model
from keras.layers.normalization import BatchNormalization
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
# instantiate model
model = Sequential()

# we can think of this chunk as the input layer
model.add(Dense(128, input_shape=(256,)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

# we can think of this chunk as the hidden layer    
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

# we can think of this chunk as the output layer
model.add(Dense(2))
model.add(BatchNormalization())
model.add(Activation('softmax'))

# setting up the optimization of our weights 
# sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
opt = optimizers.adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])


checkpointer=ModelCheckpoint(filepath="ann.hdf5",monitor = 'val_accuracy',verbose=1,save_best_only=True)

history=model.fit(
  X_train,
  to_categorical(y_train),
  epochs=500,
  batch_size=16,
  validation_data=(X_test, to_categorical(y_test)),callbacks=[checkpointer],
  )  # A binary matrix representation of the input.


Train on 600 samples, validate on 400 samples
Epoch 1/500
600/600 [==============================] - 3s 4ms/step - loss: 0.7334 - accuracy: 0.6033 - auc_20: 0.5948 - precision_20: 0.5773 - recall_20: 0.5773 - val_loss: 0.8509 - val_accuracy: 0.6175 - val_auc_20: 0.6564 - val_precision_20: 0.6047 - val_recall_20: 0.6047

Epoch 00001: val_accuracy improved from -inf to 0.61750, saving model to ann.hdf5
Epoch 2/500
600/600 [==============================] - 0s 607us/step - loss: 0.6016 - accuracy: 0.7067 - auc_20: 0.6801 - precision_20: 0.6256 - recall_20: 0.6256 - val_loss: 0.6632 - val_accuracy: 0.6900 - val_auc_20: 0.7039 - val_precision_20: 0.6500 - val_recall_20: 0.6500

Epoch 00002: val_accuracy improved from 0.61750 to 0.69000, saving model to ann.hdf5
Epoch 3/500
600/600 [==============================] - 0s 606us/step - loss: 0.5696 - accuracy: 0.7333 - auc_20: 0.7206 - precision_20: 0.6666 - recall_20: 0.6666 - val_loss: 0.6077 - val_accuracy: 0.6925 - val_auc_20: 0.7301 - val_p


Epoch 00024: val_accuracy did not improve from 0.76000
Epoch 25/500
600/600 [==============================] - 0s 591us/step - loss: 0.2741 - accuracy: 0.8850 - auc_20: 0.8496 - precision_20: 0.7768 - recall_20: 0.7768 - val_loss: 0.5702 - val_accuracy: 0.7275 - val_auc_20: 0.8508 - val_precision_20: 0.7777 - val_recall_20: 0.7777

Epoch 00025: val_accuracy did not improve from 0.76000
Epoch 26/500
600/600 [==============================] - 0s 613us/step - loss: 0.2641 - accuracy: 0.9067 - auc_20: 0.8520 - precision_20: 0.7788 - recall_20: 0.7788 - val_loss: 0.5745 - val_accuracy: 0.7275 - val_auc_20: 0.8533 - val_precision_20: 0.7799 - val_recall_20: 0.7799

Epoch 00026: val_accuracy did not improve from 0.76000
Epoch 27/500
600/600 [==============================] - 0s 603us/step - loss: 0.2646 - accuracy: 0.9083 - auc_20: 0.8545 - precision_20: 0.7810 - recall_20: 0.7810 - val_loss: 0.5718 - val_accuracy: 0.7575 - val_auc_20: 0.8557 - val_precision_20: 0.7823 - val_recall_20: 0.782

600/600 [==============================] - 0s 585us/step - loss: 0.1311 - accuracy: 0.9633 - auc_20: 0.8897 - precision_20: 0.8168 - recall_20: 0.8168 - val_loss: 0.6889 - val_accuracy: 0.7150 - val_auc_20: 0.8900 - val_precision_20: 0.8173 - val_recall_20: 0.8173

Epoch 00049: val_accuracy did not improve from 0.76000
Epoch 50/500
600/600 [==============================] - 0s 568us/step - loss: 0.1291 - accuracy: 0.9617 - auc_20: 0.8905 - precision_20: 0.8178 - recall_20: 0.8178 - val_loss: 0.6956 - val_accuracy: 0.7150 - val_auc_20: 0.8908 - val_precision_20: 0.8181 - val_recall_20: 0.8181

Epoch 00050: val_accuracy did not improve from 0.76000
Epoch 51/500
600/600 [==============================] - 0s 580us/step - loss: 0.1704 - accuracy: 0.9283 - auc_20: 0.8911 - precision_20: 0.8184 - recall_20: 0.8184 - val_loss: 0.6847 - val_accuracy: 0.7200 - val_auc_20: 0.8914 - val_precision_20: 0.8187 - val_recall_20: 0.8187

Epoch 00051: val_accuracy did not improve from 0.76000
Epoch 52/50


Epoch 00073: val_accuracy did not improve from 0.76000
Epoch 74/500
600/600 [==============================] - 0s 596us/step - loss: 0.0859 - accuracy: 0.9733 - auc_20: 0.9046 - precision_20: 0.8353 - recall_20: 0.8353 - val_loss: 0.7350 - val_accuracy: 0.7250 - val_auc_20: 0.9048 - val_precision_20: 0.8355 - val_recall_20: 0.8355

Epoch 00074: val_accuracy did not improve from 0.76000
Epoch 75/500
600/600 [==============================] - 0s 618us/step - loss: 0.0884 - accuracy: 0.9767 - auc_20: 0.9050 - precision_20: 0.8359 - recall_20: 0.8359 - val_loss: 0.7184 - val_accuracy: 0.7125 - val_auc_20: 0.9052 - val_precision_20: 0.8361 - val_recall_20: 0.8361

Epoch 00075: val_accuracy did not improve from 0.76000
Epoch 76/500
600/600 [==============================] - 0s 615us/step - loss: 0.0709 - accuracy: 0.9850 - auc_20: 0.9055 - precision_20: 0.8364 - recall_20: 0.8364 - val_loss: 0.7345 - val_accuracy: 0.7250 - val_auc_20: 0.9057 - val_precision_20: 0.8366 - val_recall_20: 0.836

600/600 [==============================] - 0s 627us/step - loss: 0.0756 - accuracy: 0.9750 - auc_20: 0.9123 - precision_20: 0.8445 - recall_20: 0.8445 - val_loss: 0.8406 - val_accuracy: 0.7200 - val_auc_20: 0.9123 - val_precision_20: 0.8446 - val_recall_20: 0.8446

Epoch 00098: val_accuracy did not improve from 0.76000
Epoch 99/500
600/600 [==============================] - 0s 621us/step - loss: 0.0860 - accuracy: 0.9717 - auc_20: 0.9124 - precision_20: 0.8448 - recall_20: 0.8448 - val_loss: 0.8117 - val_accuracy: 0.7000 - val_auc_20: 0.9125 - val_precision_20: 0.8449 - val_recall_20: 0.8449

Epoch 00099: val_accuracy did not improve from 0.76000
Epoch 100/500
600/600 [==============================] - 0s 574us/step - loss: 0.0976 - accuracy: 0.9733 - auc_20: 0.9125 - precision_20: 0.8450 - recall_20: 0.8450 - val_loss: 0.8076 - val_accuracy: 0.7175 - val_auc_20: 0.9126 - val_precision_20: 0.8451 - val_recall_20: 0.8451

Epoch 00100: val_accuracy did not improve from 0.76000
Epoch 101/

...


Epoch 00391: val_accuracy did not improve from 0.76000
Epoch 392/500
600/600 [==============================] - 0s 668us/step - loss: 0.0047 - accuracy: 0.9983 - auc_20: 0.9350 - precision_20: 0.8743 - recall_20: 0.8743 - val_loss: 1.0903 - val_accuracy: 0.7250 - val_auc_20: 0.9350 - val_precision_20: 0.8743 - val_recall_20: 0.8743

Epoch 00392: val_accuracy did not improve from 0.76000
Epoch 393/500
600/600 [==============================] - 0s 655us/step - loss: 0.0167 - accuracy: 0.9950 - auc_20: 0.9350 - precision_20: 0.8743 - recall_20: 0.8743 - val_loss: 1.0705 - val_accuracy: 0.7100 - val_auc_20: 0.9350 - val_precision_20: 0.8743 - val_recall_20: 0.8743

Epoch 00393: val_accuracy did not improve from 0.76000
Epoch 394/500
600/600 [==============================] - 0s 665us/step - loss: 0.0070 - accuracy: 1.0000 - auc_20: 0.9351 - precision_20: 0.8744 - recall_20: 0.8744 - val_loss: 0.9982 - val_accuracy: 0.7225 - val_auc_20: 0.9351 - val_precision_20: 0.8744 - val_recall_20: 0.

600/600 [==============================] - 0s 641us/step - loss: 0.0053 - accuracy: 0.9983 - auc_20: 0.9357 - precision_20: 0.8750 - recall_20: 0.8750 - val_loss: 1.1687 - val_accuracy: 0.7300 - val_auc_20: 0.9357 - val_precision_20: 0.8751 - val_recall_20: 0.8751

Epoch 00416: val_accuracy did not improve from 0.76000
Epoch 417/500
600/600 [==============================] - 0s 671us/step - loss: 0.0150 - accuracy: 0.9967 - auc_20: 0.9358 - precision_20: 0.8751 - recall_20: 0.8751 - val_loss: 1.1645 - val_accuracy: 0.7225 - val_auc_20: 0.9358 - val_precision_20: 0.8751 - val_recall_20: 0.8751

Epoch 00417: val_accuracy did not improve from 0.76000
Epoch 418/500
600/600 [==============================] - 0s 667us/step - loss: 0.0282 - accuracy: 0.9967 - auc_20: 0.9358 - precision_20: 0.8751 - recall_20: 0.8751 - val_loss: 1.2198 - val_accuracy: 0.7250 - val_auc_20: 0.9358 - val_precision_20: 0.8751 - val_recall_20: 0.8751

Epoch 00418: val_accuracy did not improve from 0.76000
Epoch 419

600/600 [==============================] - 0s 644us/step - loss: 0.0133 - accuracy: 0.9950 - auc_20: 0.9362 - precision_20: 0.8755 - recall_20: 0.8755 - val_loss: 1.0861 - val_accuracy: 0.7025 - val_auc_20: 0.9362 - val_precision_20: 0.8755 - val_recall_20: 0.8755

Epoch 00440: val_accuracy did not improve from 0.76000
Epoch 441/500
600/600 [==============================] - 0s 662us/step - loss: 0.0093 - accuracy: 0.9983 - auc_20: 0.9362 - precision_20: 0.8755 - recall_20: 0.8755 - val_loss: 1.0237 - val_accuracy: 0.7125 - val_auc_20: 0.9362 - val_precision_20: 0.8755 - val_recall_20: 0.8755

Epoch 00441: val_accuracy did not improve from 0.76000
Epoch 442/500
600/600 [==============================] - 0s 662us/step - loss: 0.0185 - accuracy: 0.9967 - auc_20: 0.9363 - precision_20: 0.8755 - recall_20: 0.8755 - val_loss: 0.9978 - val_accuracy: 0.7050 - val_auc_20: 0.9363 - val_precision_20: 0.8755 - val_recall_20: 0.8755

Epoch 00442: val_accuracy did not improve from 0.76000
Epoch 443


Epoch 00464: val_accuracy did not improve from 0.76000
Epoch 465/500
600/600 [==============================] - 0s 623us/step - loss: 0.0715 - accuracy: 0.9917 - auc_20: 0.9368 - precision_20: 0.8758 - recall_20: 0.8758 - val_loss: 1.0522 - val_accuracy: 0.7150 - val_auc_20: 0.9368 - val_precision_20: 0.8758 - val_recall_20: 0.8758

Epoch 00465: val_accuracy did not improve from 0.76000
Epoch 466/500
600/600 [==============================] - 0s 675us/step - loss: 0.0075 - accuracy: 0.9983 - auc_20: 0.9368 - precision_20: 0.8758 - recall_20: 0.8758 - val_loss: 1.0229 - val_accuracy: 0.7150 - val_auc_20: 0.9369 - val_precision_20: 0.8759 - val_recall_20: 0.8759

Epoch 00466: val_accuracy did not improve from 0.76000
Epoch 467/500
600/600 [==============================] - 0s 696us/step - loss: 0.0128 - accuracy: 0.9950 - auc_20: 0.9369 - precision_20: 0.8759 - recall_20: 0.8759 - val_loss: 1.0366 - val_accuracy: 0.7150 - val_auc_20: 0.9369 - val_precision_20: 0.8759 - val_recall_20: 0.

600/600 [==============================] - ETA: 0s - loss: 0.0438 - accuracy: 0.9932 - auc_20: 0.9373 - precision_20: 0.8764 - recall_20: 0.87 - 0s 659us/step - loss: 0.0434 - accuracy: 0.9933 - auc_20: 0.9373 - precision_20: 0.8764 - recall_20: 0.8764 - val_loss: 1.0593 - val_accuracy: 0.7075 - val_auc_20: 0.9374 - val_precision_20: 0.8764 - val_recall_20: 0.8764

Epoch 00489: val_accuracy did not improve from 0.76000
Epoch 490/500
600/600 [==============================] - 0s 660us/step - loss: 0.0235 - accuracy: 0.9933 - auc_20: 0.9374 - precision_20: 0.8764 - recall_20: 0.8764 - val_loss: 1.0286 - val_accuracy: 0.6975 - val_auc_20: 0.9374 - val_precision_20: 0.8764 - val_recall_20: 0.8764

Epoch 00490: val_accuracy did not improve from 0.76000
Epoch 491/500
600/600 [==============================] - 0s 672us/step - loss: 0.0434 - accuracy: 0.9933 - auc_20: 0.9374 - precision_20: 0.8764 - recall_20: 0.8764 - val_loss: 1.0099 - val_accuracy: 0.7075 - val_auc_20: 0.9374 - val_precisio

In [64]:
model.load_weights('ann.hdf5')

In [65]:
results=model.evaluate(X_test,to_categorical(y_test),batch_size=1)
print('test loss', results[0])
print('test acc', results[1])
print('test auc', results[2])
print('test precision', results[3])
print('test recall', results[4])

400/400 [==============================] - 1s 2ms/step
test loss 0.5407273634755984
test acc 0.7599999904632568
test auc 0.9375243186950684
test precision 0.8763954043388367
test recall 0.8763954043388367


References:
1. https://scikit-learn.org/stable/
2. https://www.analyticsvidhya.com/blog/2017/09/common-machine-learning-algorithms/

**Content prepared Principal Coordinator: Prof. Aparajita Ojha Local Coordinator: Dr. Ayan Seal Tutorial Assistant: Mr. Samir Jain and Mr. Nishant Khare (Phd Students)**

**E&ICT Academy PDPM IIITDM Jabalpur**